In [1]:
import csv
import pandas as pd
import time
import sklearn
import random
import sqlite3
from estnltk import Text
from estnltk_neural.taggers import StanzaSyntaxTagger
from estnltk.taggers import NerTagger
import json
from estnltk.converters import text_to_json

In [2]:
from estnltk import Layer
from estnltk.taggers import Tagger
from collections import defaultdict
from collections import OrderedDict
import copy

In [3]:
def get_ner(ner_layer, word_layer, span):
        nertag = None
        if len(ner_layer) > 0:
            word = word_layer.get(span)
            for n in ner_layer:
                for part in n:
                    if part==word:
                        nertag=n.nertag
        if nertag:
            return nertag
        return 'OTHER'
    
def get_POS(word_layer, span):
    infinite_verb_forms = ['da', 'des', 'ma', 'maks', 'mas', 'mast', 'mata', 'nud', 'tav', 'tud', 'v']
    # if POS is ambiguous, only unique tags are kept, e.g. ['V', 'A', 'A'] -> ['V', 'A']
    pos_list = []
    word = word_layer.get(span)
    for i in range(len(word.morph_analysis['partofspeech'])):
        if word.morph_analysis['partofspeech'][i] == 'V':
            if word.morph_analysis['form'][i] in infinite_verb_forms:
                pos_list.append('V_inf')
            elif word.form[i] == 'neg':
                pos_list.append('V_neg')
            else:
                pos_list.append('V_fin')
        else:
            pos_list.append(word.morph_analysis['partofspeech'][i])
    
    if len(pos_list) > 1:
        char_unique = [char for indx, char in enumerate(pos_list) if char not in pos_list[:indx]]
        if len(char_unique) < 2:
            return char_unique[0]
        return '|'.join(char_unique)
    return pos_list[0]

In [17]:
class PhrasePatternTagger(Tagger):
    """Tags phrases that match given syntax and part-of-speech pattern rules, and their corresponding patterns.""" 
    
    conf_param = ['rules_file', 'ruleset_map']
    
    def __init__(self, rules_file: str,
                       output_layer='phrase_patterns',
                       morph_analysis_layer='morph_analysis',
                       words_layer='words',
                       syntax_layer='stanza_syntax',
                       ner_layer='ner'):
        
        self.input_layers = [morph_analysis_layer, words_layer, syntax_layer, ner_layer]
        self.output_layer = output_layer
        self.output_attributes = ['extraction_pattern', 'ner_pattern', 'pattern_id', 'score', 'phrase_pattern_id', 'phrase_class']
        self.rules_file = rules_file

        ruleset_map = defaultdict(list)
        
        with open(rules_file, encoding='UTF-8') as csv_file:
            reader = csv.DictReader(csv_file)
            for row in reader:
                info = [row['ID'], row['POS_pattern'], row['NER_pattern']]
                ruleset_map[row['tree']].append(info)
                
        self.ruleset_map = ruleset_map

    def _make_layer_template(self):
        layer = Layer(name=self.output_layer,
                      text_object=None,
                      attributes=self.output_attributes,
                      enveloping=self.input_layers[1],
                      ambiguous=True)
        return layer
        
    def _make_layer(self, text, layers, status):
        layer = self._make_layer_template()
        layer.text_object = text
        
        for i in range(len(layers[self.input_layers[2]])): # Iterate over 'stanza_syntax' layer
            pattern_spans = []    
            ids = []

            pattern_spans.append(layers[self.input_layers[2]][i])
            ids.append([layers[self.input_layers[2]][i]['id'], layers[self.input_layers[2]][i]['head']])
                
            for j in range(i + 1, len(layers[self.input_layers[2]])):
                tree = []
                pos = []
                ner = []
                for k in range(len(pattern_spans)):
                    if layers[self.input_layers[2]][j] not in pattern_spans:
                        if layers[self.input_layers[2]][j] in pattern_spans[k]['children'] or pattern_spans[k] in layers[self.input_layers[2]][j]['children'] or layers[self.input_layers[2]][j]['parent_span'] != None and layers[self.input_layers[2]][j]['parent_span'] == pattern_spans[k]['parent_span']:
                            pattern_spans.append(layers[self.input_layers[2]][j])
                            ids.append([layers[self.input_layers[2]][j]['id'], layers[self.input_layers[2]][j]['head']])
                
                # fixing word and head ID values
                ids_for_pattern = copy.deepcopy(ids)
                for k in range(len(ids_for_pattern)):
                    temp = ids_for_pattern[k][0]
                    ids_for_pattern[k][0] = k+1
                    for l in range(len(ids)):
                        if ids[l][1] == temp:
                            ids_for_pattern[l][1] = ids_for_pattern[k][0]
            
                word_ids = [word_id[0] for word_id in ids_for_pattern]
                for k in range(len(ids_for_pattern)):
                    if ids_for_pattern[k][0] == ids_for_pattern[k][1]:
                        ids_for_pattern[k][1] = 0
                    elif ids_for_pattern[k][1] not in word_ids:
                        ids_for_pattern[k][1] = 0
                
                # finding the root of current pattern and setting its deprel value as such
                for k in range(len(pattern_spans)):
                    deprel = pattern_spans[k].deprel
                    if ids_for_pattern[k][1] == 0 and deprel != 'root':
                        deprel = 'root'
                    tree.append([str(ids_for_pattern[k][0]), str(ids_for_pattern[k][1]), deprel])
                    # POS-tag is taken from morph_analysis layer
                    pos.append(get_POS(layers[self.input_layers[1]], pattern_spans[k]))
                    # nertag is taken from ner layer
                    ner.append(get_ner(layers[self.input_layers[-1]], layers[self.input_layers[1]], pattern_spans[k]))                     
                    
                pattern = [" ".join(word_info) for word_info in tree]
                #print(pattern)
                # check if tree pattern exists in ruleset map
                if ",".join(pattern) in self.ruleset_map.keys():
                    #print(pattern, 'yes')
                    pos_pattern = "-".join(pos)
                    ner_pattern = "-".join(ner)
                    # check if POS-sequence and NER-sequence exist in ruleset map with given tree pattern
                    for el in self.ruleset_map[",".join(pattern)]:
                        #print(el[1], pos_pattern, el[2], ner_pattern)
                        if el[1] == pos_pattern and el[2] == ner_pattern:
                            #print(pattern, 'yesyes')
                            # add annotation
                            layer.add_annotation([span.base_span for span in pattern_spans], 
                                                 extraction_pattern=",".join([",".join(pattern), pos_pattern]),
                                                 ner_pattern=ner_pattern,
                                                 pattern_id=el[0],
                                                 score=None,
                                                 phrase_pattern_id=None,
                                                 phrase_class=None)
                            #tree = []
                            #pos = []
                            #ner = []

                #if j == len(layers[self.input_layers[2]]) - 1:
                    #pattern_spans = []
                    #ids = []
                    #pos = []
                    #ner = []
                    #break             
                
        return layer
    


In [18]:
pattern_tagger = PhrasePatternTagger(rules_file='indicator_patterns_ner_tree_pos_updated.csv')
pattern_tagger

PhrasePatternTagger(input_layers=('morph_analysis', 'words', 'stanza_syntax', 'ner'), output_layer=phrase_patterns, output_attributes=('extraction_pattern', 'ner_pattern', 'pattern_id', 'score', 'phrase_pattern_id', 'phrase_class'), rules_file=indicator_patterns_ner_tree_pos_updated.csv, ruleset_map=defaultdict(<class 'list'>, {'string': [['int64', 'string', 'string']], '1 2 nmo ..., type: <class 'collections.defaultdict'>, length: 7)

In [6]:
stanza_tagger = StanzaSyntaxTagger(input_type='morph_analysis', input_morph_layer='morph_analysis',
                                   add_parent_and_children=True)

In [7]:
ner_tagger = NerTagger()

In [141]:
test_sent1 = Text('Kohtuasjade lahendamine avaldusega').tag_layer()
stanza_tagger.tag(test_sent1)
ner_tagger.tag(test_sent1)
pattern_tagger.tag(test_sent1)

Text(text='Kohtuasjade lahendamine avaldusega')

In [142]:
test_sent1.phrase_patterns

Layer(name='phrase_patterns', attributes=('extraction_pattern', 'ner_pattern', 'pattern_id', 'score', 'phrase_pattern_id', 'phrase_class'), spans=SL[EnvelopingSpan(['Kohtuasjade', 'lahendamine'], [{'extraction_pattern': '1 2 nmod,2 0 root,S-S', 'ner_pattern': 'OTHER-OTHER', 'pattern_id': '1', 'score': None, 'phrase_pattern_id': None, 'phrase_class': None}]),
EnvelopingSpan(['Kohtuasjade', 'lahendamine', 'avaldusega'], [{'extraction_pattern': '1 2 nmod,2 0 root,3 2 nmod,S-S-S', 'ner_pattern': 'OTHER-OTHER-OTHER', 'pattern_id': '129', 'score': None, 'phrase_pattern_id': None, 'phrase_class': None}])])

In [143]:
test_sent2 = Text('suhete direktor Piret Mürk').tag_layer()
stanza_tagger.tag(test_sent2)
ner_tagger.tag(test_sent2)
pattern_tagger.tag(test_sent2)

Text(text='suhete direktor Piret Mürk')

In [144]:
test_sent2.phrase_patterns

Layer(name='phrase_patterns', attributes=('extraction_pattern', 'ner_pattern', 'pattern_id', 'score', 'phrase_pattern_id', 'phrase_class'), spans=SL[EnvelopingSpan(['suhete', 'direktor'], [{'extraction_pattern': '1 2 nmod,2 0 root,S-S', 'ner_pattern': 'OTHER-OTHER', 'pattern_id': '1', 'score': None, 'phrase_pattern_id': None, 'phrase_class': None}]),
EnvelopingSpan(['suhete', 'direktor', 'Piret', 'Mürk'], [{'extraction_pattern': '1 2 nmod,2 0 root,3 2 appos,4 3 flat,S-S-H-H', 'ner_pattern': 'OTHER-OTHER-OTHER-OTHER', 'pattern_id': '156', 'score': None, 'phrase_pattern_id': None, 'phrase_class': None}])])

In [145]:
test_df = pd.read_pickle('phrase_examples_filtered_sub10000/atomic_phrases.pkl')

In [146]:
test_df

,phrase,phrase_length,document_creation_time,sentence_id,document_id,sentence_startend,subcorpus,phrase_type,phrase_start_end,has_ner_netity,...,pos_sequence,graph,graph_code,graph_code_pos,graph_code_pos_ner_timex,pos_sequence_verb_info,graph_verb_info,graph_code_verb_info,graph_code_pos_verb_info,graph_code_pos_ner_timex_verb_info
0,Text(text='külmakuudega'),1,2024-03-10T00:43,5858921,312319,"(0, 226)",aja_EPL,obl_phrase,"(6, 18)",0,...,S,"(1, 0)","((0, 1, root))","(S,(0, 1, root))","(0,S,(0, 1, root))",S,"(1, 0)","((0, 1, root))","(S,(0, 1, root))","(0,S,(0, 1, root))"
1,Text(text='libedavastast graniidipuru'),2,2024-03-10T00:43,5858921,312319,"(0, 226)",aja_EPL,nsubj_phrase,"(22, 48)",0,...,A-S,"(1, 2, 0)","((0, 2, root),(2, 1, amod))","(S-A,(0, 2, root),(2, 1, amod))","(0-0,S-A,(0, 2, root),(2, 1, amod))",A-S,"(1, 2, 0)","((0, 2, root),(2, 1, amod))","(S-A,(0, 2, root),(2, 1, amod))","(0-0,S-A,(0, 2, root),(2, 1, amod))"
2,Text(text='juba vallid'),2,2024-03-10T00:43,5858921,312319,"(0, 226)",aja_EPL,nsubj_phrase,"(114, 125)",0,...,D-S,"(1, 2, 0)","((0, 2, root),(2, 1, advmod))","(S-D,(0, 2, root),(2, 1, advmod))","(0-0,S-D,(0, 2, root),(2, 1, advmod))",D-S,"(1, 2, 0)","((0, 2, root),(2, 1, advmod))","(S-D,(0, 2, root),(2, 1, advmod))","(0-0,S-D,(0, 2, root),(2, 1, advmod))"
3,Text(text='tänavapuhastajad'),1,2024-03-10T00:43,5858921,312319,"(0, 226)",aja_EPL,nsubj_phrase,"(139, 155)",0,...,S,"(1, 0)","((0, 1, root))","(S,(0, 1, root))","(0,S,(0, 1, root))",S,"(1, 0)","((0, 1, root))","(S,(0, 1, root))","(0,S,(0, 1, root))"
4,Text(text='külma'),1,2024-03-10T00:43,5858921,312319,"(0, 226)",aja_EPL,nsubj_phrase,"(211, 216)",0,...,A,"(1, 0)","((0, 1, root))","(A,(0, 1, root))","(0,A,(0, 1, root))",A,"(1, 0)","((0, 1, root))","(A,(0, 1, root))","(0,A,(0, 1, root))"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36060,Text(text='kooridele'),1,2024-03-10T01:35,4265502,224097,"(1430, 1538)",aja_EPL,obl_phrase,"(36, 45)",0,...,S,"(1, 0)","((0, 1, root))","(S,(0, 1, root))","(0,S,(0, 1, root))",S,"(1, 0)","((0, 1, root))","(S,(0, 1, root))","(0,S,(0, 1, root))"
36061,Text(text='mis'),1,2024-03-10T01:35,4265502,224097,"(1430, 1538)",aja_EPL,nsubj_phrase,"(59, 62)",0,...,P,"(1, 0)","((0, 1, root))","(P,(0, 1, root))","(0,P,(0, 1, root))",P,"(1, 0)","((0, 1, root))","(P,(0, 1, root))","(0,P,(0, 1, root))"
36062,Text(text='ka koorijuhi palgamäära'),3,2024-03-10T01:35,4265502,224097,"(1430, 1538)",aja_EPL,obj_phrase,"(83, 106)",0,...,D-S-S,"(1, 3, 2, 0)","((0, 3, root),(3, 1, advmod),(3, 2, nmod))","(S-D-S,(0, 3, root),(3, 1, advmod),(3, 2, nmod))","(0-0-0,S-D-S,(0, 3, root),(3, 1, advmod),(3, 2, nmod))",D-S-S,"(1, 3, 2, 0)","((0, 3, root),(3, 1, advmod),(3, 2, nmod))","(S-D-S,(0, 3, root),(3, 1, advmod),(3, 2, nmod))","(0-0-0,S-D-S,(0, 3, root),(3, 1, advmod),(3, 2, nmod))"
36063,Text(text='Laulupeo'),1,2024-03-10T01:35,4265502,224097,"(1430, 1538)",aja_EPL,nmod_phrase,"(0, 8)",0,...,H,"(1, 0)","((0, 1, root))","(H,(0, 1, root))","(0,H,(0, 1, root))",H,"(1, 0)","((0, 1, root))","(H,(0, 1, root))","(0,H,(0, 1, root))"


In [147]:
test_500 = []

for idx, row in test_df.iterrows():
    if row['phrase_length'] > 1:
        test_500.append(row['phrase'])
    if len(test_500) == 500:
        break

In [148]:
for phrase in test_500:
    pattern_tagger.tag(phrase)

In [149]:
n_found = 0
for phrase in test_500:
    try:
        if len(phrase.phrase_patterns) > 0 and len(phrase.ner) > 0:
            display(phrase)
            display(phrase.phrase_patterns)
            n_found+=1
    except:
        break
print(n_found)

Text(text='ETA andmetel')

Layer(name='phrase_patterns', attributes=('extraction_pattern', 'ner_pattern', 'pattern_id', 'score', 'phrase_pattern_id', 'phrase_class'), spans=SL[EnvelopingSpan(['ETA', 'andmetel'], [{'extraction_pattern': '1 2 nmod,2 0 root,Y-S', 'ner_pattern': 'ORG-OTHER', 'pattern_id': '20', 'score': None, 'phrase_pattern_id': None, 'phrase_class': None}])])

Text(text='Riia kesklinnas asuva kohviku kõrval')

Layer(name='phrase_patterns', attributes=('extraction_pattern', 'ner_pattern', 'pattern_id', 'score', 'phrase_pattern_id', 'phrase_class'), spans=SL[EnvelopingSpan(['Riia', 'kesklinnas'], [{'extraction_pattern': '1 2 nmod,2 0 root,H-S', 'ner_pattern': 'LOC-OTHER', 'pattern_id': '11', 'score': None, 'phrase_pattern_id': None, 'phrase_class': None}]),
EnvelopingSpan(['kesklinnas', 'asuva', 'kohviku'], [{'extraction_pattern': '1 2 obl,2 3 acl,3 0 root,S-A-S', 'ner_pattern': 'OTHER-OTHER-OTHER', 'pattern_id': '94', 'score': None, 'phrase_pattern_id': None, 'phrase_class': None}])])

Text(text='Eesti sõjaväes')

...

Text(text='Kaitsepolitsei süüdistuse järgi')

Layer(name='phrase_patterns', attributes=('extraction_pattern', 'ner_pattern', 'pattern_id', 'score', 'phrase_pattern_id', 'phrase_class'), spans=SL[EnvelopingSpan(['Kaitsepolitsei', 'süüdistuse'], [{'extraction_pattern': '1 2 nmod,2 0 root,H-S', 'ner_pattern': 'ORG-OTHER', 'pattern_id': '14', 'score': None, 'phrase_pattern_id': None, 'phrase_class': None}])])

Text(text='Eesti meistriks')

Layer(name='phrase_patterns', attributes=('extraction_pattern', 'ner_pattern', 'pattern_id', 'score', 'phrase_pattern_id', 'phrase_class'), spans=SL[EnvelopingSpan(['Eesti', 'meistriks'], [{'extraction_pattern': '1 2 nmod,2 0 root,H-S', 'ner_pattern': 'LOC-OTHER', 'pattern_id': '11', 'score': None, 'phrase_pattern_id': None, 'phrase_class': None}])])

25


In [10]:
# older version output
n_found = 0
for phrase in test_500:
    try:
        if len(phrase.phrase_patterns) > 0 and len(phrase.ner) > 0:
            display(phrase)
            display(phrase.phrase_patterns)
            n_found+=1
    except:
        break
print(n_found)

Text(text='ETA andmetel')

layer name,attributes,parent,enveloping,ambiguous,span count
phrase_patterns,"extraction_pattern, pattern_id, score, phrase_pattern_id, phrase_class",None,words,True,1
text,extraction_pattern,pattern_id,score,phrase_pattern_id,phrase_class
"['ETA', 'andmetel']","1 2 nmod,2 0 root,Y-S,ORG-OTHER",20,None,None,None


Text(text='Riia kesklinnas asuva kohviku kõrval')

layer name,attributes,parent,enveloping,ambiguous,span count
phrase_patterns,"extraction_pattern, pattern_id, score, phrase_pattern_id, phrase_class",None,words,True,1
text,extraction_pattern,pattern_id,score,phrase_pattern_id,phrase_class
"['Riia', 'kesklinnas']","1 2 nmod,2 0 root,H-S,LOC-OTHER",11,None,None,None


Text(text='Eesti sõjaväes')

...

Text(text='Kaitsepolitsei süüdistuse järgi')

layer name,attributes,parent,enveloping,ambiguous,span count
phrase_patterns,"extraction_pattern, pattern_id, score, phrase_pattern_id, phrase_class",None,words,True,1
text,extraction_pattern,pattern_id,score,phrase_pattern_id,phrase_class
"['Kaitsepolitsei', 'süüdistuse']","1 2 nmod,2 0 root,H-S,ORG-OTHER",14,None,None,None


Text(text='Eesti meistriks')

layer name,attributes,parent,enveloping,ambiguous,span count
phrase_patterns,"extraction_pattern, pattern_id, score, phrase_pattern_id, phrase_class",None,words,True,1
text,extraction_pattern,pattern_id,score,phrase_pattern_id,phrase_class
"['Eesti', 'meistriks']","1 2 nmod,2 0 root,H-S,LOC-OTHER",11,None,None,None


25


In [20]:
con = sqlite3.connect('correct_noun_phrases.db')

In [21]:
cur = con.cursor()
cur.execute("SELECT * FROM correct_phrase_patterns")
rows = cur.fetchall()

In [22]:
tagged_phrases = defaultdict(list)

random.shuffle(rows)

start = time.time()
tagged_total = 0

for row in rows:
    text = Text(row[8]).tag_layer('morph_analysis')
    ner_tagger.tag(text)
    stanza_tagger.tag(text)
    pattern_tagger.tag(text)
    if len(text.phrase_patterns) > 0:
        tagged_total+=len(text.phrase_patterns)
        for pattern in text.phrase_patterns:
            # at the moment, up to 100 samples for each pattern are kept and later saved in database
            if len(tagged_phrases[pattern['extraction_pattern'][0]]) < 100:
                tagged_phrases[pattern['extraction_pattern'][0]].append(text)
                
print(f"{len(rows)} fraasi märgendamiseks kulus PhrasePatternTaggeril {time.time()-start} sekundit")
print(f"Tagger leidis {tagged_total} fraasi")

21492 fraasi märgendamiseks kulus PhrasePatternTaggeril 814.3563294410706 sekundit
Tagger leidis 21703 fraasi


In [24]:
for el in tagged_phrases:
    print(el, tagged_phrases[el])
    for phrase in tagged_phrases[el]:
        for p in phrase.phrase_patterns:
            p_spans = []
            for span in p:
                p_spans.append(span.text)
            print(p_spans)
    break

1 2 amod,2 0 root,U-S [Text(text='Maitsvaimaks kõhutäiteks'), Text(text='parimaks näitlejaks'), Text(text='väikseimat hammakat'), Text(text='kõrgeima riigivõimu'), Text(text='parimaks korvikütiks'), Text(text='loodepoolseimast neemest'), Text(text='populaarseimad vabaõhulavad'), Text(text='parimaks korvikütiks'), Text(text='kõrgeima riigivõimu'), Text(text='vähimagi kahtluse'), Text(text='parimal viisil'), Text(text='Parimad ettevõtted'), Text(text='parimaks väravavahiks'), Text(text='moodsaima õhutõrjetehnika'), Text(text='oodatuima kohtumise'), Text(text='suurimat massi'), Text(text='suurim sõidukiirus'), Text(text='keerukaima toote'), Text(text='lähimasse linna'), Text(text='soosituima pallimängu'), Text(text='Parim film'), Text(text='suurimaid imetajaid'), Text(text='kiireim mees'), Text(text='vähimagi minevikutaagata'), Text(text='kõrgeima juhina'), Text(text='tugevaimatel registritel'), Text(text='Suurima pagulaskirjastuse'), Text(text='Suurima pagulaskirjastuse'), Text(text='väh

In [23]:
for el in tagged_phrases:
    print(el, len(tagged_phrases[el]))

1 2 amod,2 0 root,O-S 100
1 2 amod,2 0 root,A-S 100
1 0 root,2 1 flat,3 1 flat,H-H-Y 19
1 2 nmod,2 0 root,S-S 100
1 2 obl,2 3 acl,3 0 root,S-A|V_inf-S 100
1 2 nmod,2 0 root,P-S 100
1 2 nmod,2 0 root,H-H 100
1 2 amod,2 0 root,P|O-S 100
1 2 nmod,2 0 root,H-S 100
1 2 nmod,2 0 root,3 2 appos,4 3 flat,S-S-H-H 100
1 2 nmod,2 0 root,Y-S 100
1 2 obl,2 3 acl,3 0 root,S-A-S 100
1 2 nmod,2 0 root,3 2 nmod,S-S-S 100
1 2 amod,2 0 root,C-S 100
1 2 amod,2 0 root,H-S 53
1 2 nmod,2 0 root,3 2 nmod,P-S-P 4
1 0 root,2 1 flat,3 1 flat,H-H-H 64
1 2 amod,2 0 root,P-S 100
1 2 amod,2 0 root,U-S 100
1 0 root,2 1 flat,3 1 flat,H-Y-H 15
1 2 amod,2 0 root,S-S 21
1 2 amod,2 0 root,G-S 100
1 2 nmod,2 0 root,3 2 appos,4 3 flat,H-S-H-H 100
1 2 nmod,2 0 root,3 2 nmod,S-S-H 31
1 2 obl,2 3 acl,3 0 root,S-V_inf|A-H 21
1 2 nmod,2 0 root,Y-N 40
1 2 obl,2 3 acl,3 0 root,H-A-S 51
1 2 nmod,2 0 root,3 2 nmod,Y-S-S 7
1 2 nmod,2 0 root,P-A 4
1 2 amod,2 0 root,O-N 3
1 2 nmod,2 0 root,P-V_fin 2
1 2 obl,2 3 acl,3 0 root,A-A-S 3
1 2

In [24]:
con.close()

In [14]:
ruleset_map = defaultdict(list)
        
with open('indicator_patterns_ner_tree_pos_updated.csv', encoding='UTF-8') as csv_file:
    reader = csv.DictReader(csv_file)
    for row in reader:
        info = [row['ID'], row['POS_pattern'], row['NER_pattern']]
        ruleset_map[row['tree']].append(info)

In [15]:
len(ruleset_map)

7

In [16]:
for el in ruleset_map:
    print(el)

string
1 2 nmod,2 0 root
1 2 amod,2 0 root
1 2 obl,2 3 acl,3 0 root
1 2 nmod,2 0 root,3 2 nmod
1 2 nmod,2 0 root,3 2 appos,4 3 flat
1 0 root,2 1 flat,3 1 flat


In [53]:
for el in tagged_phrases:
    for phrase in tagged_phrases[el]:
        for p in phrase.phrase_patterns:
            raw = []
            for span in p:
                morph_word = phrase.morph_analysis.get(span)
                # first lemma
                raw.append(morph_word.lemma[0])
            print(p['ner_pattern'][0])
            print(p['pattern_id'][0])
            print(p['extraction_pattern'][0])
            print(raw)
            break

OTHER-OTHER
72
1 2 amod,2 0 root,O-S
['20.', 'august']
OTHER-OTHER
72
1 2 amod,2 0 root,O-S
['1974.', 'aasta']
OTHER-OTHER
72
1 2 amod,2 0 root,O-S
['19.', 'sajand']
OTHER-OTHER
72
1 2 amod,2 0 root,O-S
['1976.', 'aasta']
OTHER-OTHER
72
1 2 amod,2 0 root,O-S
['kolmekümnes', 'aasta']
OTHER-OTHER
72
1 2 amod,2 0 root,O-S
['1994.', 'aasta']
OTHER-OTHER
72
1 2 amod,2 0 root,O-S
['esimene', 'erakond']
OTHER-OTHER
72
1 2 amod,2 0 root,O-S
['viies', 'mäng']
OTHER-OTHER
72
1 2 amod,2 0 root,O-S
['1996.', 'aasta']
OTHER-OTHER
72
1 2 amod,2 0 root,O-S
['üheksas', 'kaotus']
OTHER-OTHER
72
1 2 amod,2 0 root,O-S
['2001.', 'aasta']
OTHER-OTHER
72
1 2 amod,2 0 root,O-S
['seitsmes', 'festivaliaasta']
OTHER-OTHER
72
1 2 amod,2 0 root,O-S
['2000.', 'aasta']
OTHER-OTHER
72
1 2 amod,2 0 root,O-S
['1995.', 'aasta']
OTHER-OTHER
72
1 2 amod,2 0 root,O-S
['2000.', 'aasta']
OTHER-OTHER
72
1 2 amod,2 0 root,O-S
['23.', 'juuni']
OTHER-OTHER
72
1 2 amod,2 0 root,O-S
['1988.', 'aasta']
OTHER-OTHER
72
1 2 amod,2 0 

In [54]:
con = sqlite3.connect("tagged_noun_phrases.db")
cur = con.cursor()
cur.execute('pragma encoding=UTF8')
cur.execute("CREATE TABLE tagged_phrases(ID INTEGER PRIMARY KEY, extraction_pattern TEXT, ner_pattern TEXT, pattern_id INTEGER, raw_text TEXT, parent_phrase TEXT)")

In [55]:
for el in tagged_phrases:
    for i in range(len(tagged_phrases[el])):
        phrase_json = text_to_json(tagged_phrases[el][i])
        for phrase in tagged_phrases[el][i].phrase_patterns:
            p_lemmas = []
            for span in phrase:
                morph_word = tagged_phrases[el][i].morph_analysis.get(span)
                # first lemma is always chosen
                p_lemmas.append(morph_word.lemma[0])
            raw_text = ' '.join([l for l in p_lemmas])
            cur.execute("""INSERT INTO tagged_phrases
                                    (extraction_pattern, ner_pattern, pattern_id, raw_text, parent_phrase)
                                    VALUES (?, ?, ?, ?, ?);""", (phrase['extraction_pattern'][0], phrase['ner_pattern'][0], phrase['pattern_id'][0], raw_text, phrase_json))
    
            con.commit()

con.close()

In [174]:
df_10000 = pd.read_pickle('data_subset_10000.pkl')

In [167]:
df_10000

,phrase,phrase_length,document_creation_time,sentence_id,document_id,sentence_startend,subcorpus,phrase_type,phrase_start_end,has_ner_netity,...,pos_sequence,graph,graph_code,graph_code_pos,graph_code_pos_ner_timex,pos_sequence_verb_info,graph_verb_info,graph_code_verb_info,graph_code_pos_verb_info,graph_code_pos_ner_timex_verb_info
0,Text(text='külmakuudega'),1,2024-03-10T00:43,5858921,312319,"(0, 226)",aja_EPL,obl_phrase,"(6, 18)",0,...,S,"(1, 0)","((0, 1, root))","(S,(0, 1, root))","(0,S,(0, 1, root))",S,"(1, 0)","((0, 1, root))","(S,(0, 1, root))","(0,S,(0, 1, root))"
1,Text(text='neist teede äärtesse'),3,2024-03-10T00:43,5858921,312319,"(0, 226)",aja_EPL,obl_phrase,"(84, 104)",0,...,P-S-S,"(1, 2, 3, 0)","((0, 3, root),(2, 1, det),(3, 2, nmod))","(S-P-S,(0, 3, root),(2, 1, det),(3, 2, nmod))","(0-0-0,S-P-S,(0, 3, root),(2, 1, det),(3, 2, nmod))",P-S-S,"(1, 2, 3, 0)","((0, 3, root),(2, 1, det),(3, 2, nmod))","(S-P-S,(0, 3, root),(2, 1, det),(3, 2, nmod))","(0-0-0,S-P-S,(0, 3, root),(2, 1, det),(3, 2, nmod))"
2,Text(text='libedavastast graniidipuru'),2,2024-03-10T00:43,5858921,312319,"(0, 226)",aja_EPL,nsubj_phrase,"(22, 48)",0,...,A-S,"(1, 2, 0)","((0, 2, root),(2, 1, amod))","(S-A,(0, 2, root),(2, 1, amod))","(0-0,S-A,(0, 2, root),(2, 1, amod))",A-S,"(1, 2, 0)","((0, 2, root),(2, 1, amod))","(S-A,(0, 2, root),(2, 1, amod))","(0-0,S-A,(0, 2, root),(2, 1, amod))"
3,Text(text='juba vallid'),2,2024-03-10T00:43,5858921,312319,"(0, 226)",aja_EPL,nsubj_phrase,"(114, 125)",0,...,D-S,"(1, 2, 0)","((0, 2, root),(2, 1, advmod))","(S-D,(0, 2, root),(2, 1, advmod))","(0-0,S-D,(0, 2, root),(2, 1, advmod))",D-S,"(1, 2, 0)","((0, 2, root),(2, 1, advmod))","(S-D,(0, 2, root),(2, 1, advmod))","(0-0,S-D,(0, 2, root),(2, 1, advmod))"
4,Text(text='tänavapuhastajad'),1,2024-03-10T00:43,5858921,312319,"(0, 226)",aja_EPL,nsubj_phrase,"(139, 155)",0,...,S,"(1, 0)","((0, 1, root))","(S,(0, 1, root))","(0,S,(0, 1, root))",S,"(1, 0)","((0, 1, root))","(S,(0, 1, root))","(0,S,(0, 1, root))"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44990,Text(text='mis'),1,2024-03-10T01:35,4265502,224097,"(1430, 1538)",aja_EPL,nsubj_phrase,"(59, 62)",0,...,P,"(1, 0)","((0, 1, root))","(P,(0, 1, root))","(0,P,(0, 1, root))",P,"(1, 0)","((0, 1, root))","(P,(0, 1, root))","(0,P,(0, 1, root))"
44991,"Text(text='kategooria , mis määrab edaspidiseks ka koorijuhi palgamäära')",8,2024-03-10T01:35,4265502,224097,"(1430, 1538)",aja_EPL,obj_phrase,"(46, 106)",0,...,S-Z-P-V-A-D-S-S,"(1, 0, 2, 4, 3, 5, 6, 8, 7)","((0, 1, root),(1, 4, acl:relcl),(4, 2, punct),(4, 3, nsubj),(4, 5, xcomp),(4, 8, obj),(8, 6, advmod),(8, 7, nmod))","(S-V-Z-P-A-S-D-S,(0, 1, root),(1, 4, acl:relcl),(4, 2, punct),(4, 3, nsubj),(4, 5, xcomp),(4, 8, obj),(8, 6, advmod),(8, 7, nmod))","(0-0-0-0-0-0-0-0,S-V-Z-P-A-S-D-S,(0, 1, root),(1, 4, acl:relcl),(4, 2, punct),(4, 3, nsubj),(4, 5, xcomp),(4, 8, obj),(8, 6, advmod),(8, 7, nmod))",S-Z-P-V_fin-A-D-S-S,"(1, 0, 2, 4, 3, 5, 6, 8, 7)","((0, 1, root),(1, 4, acl:relcl),(4, 2, punct),(4, 3, nsubj),(4, 5, xcomp),(4, 8, obj),(8, 6, advmod),(8, 7, nmod))","(S-V_fin-Z-P-A-S-D-S,(0, 1, root),(1, 4, acl:relcl),(4, 2, punct),(4, 3, nsubj),(4, 5, xcomp),(4, 8, obj),(8, 6, advmod),(8, 7, nmod))","(0-0-0-0-0-0-0-0,S-V_fin-Z-P-A-S-D-S,(0, 1, root),(1, 4, acl:relcl),(4, 2, punct),(4, 3, nsubj),(4, 5, xcomp),(4, 8, obj),(8, 6, advmod),(8, 7, nmod))"
44992,Text(text='ka koorijuhi palgamäära'),3,2024-03-10T01:35,4265502,224097,"(1430, 1538)",aja_EPL,obj_phrase,"(83, 106)",0,...,D-S-S,"(1, 3, 2, 0)","((0, 3, root),(3, 1, advmod),(3, 2, nmod))","(S-D-S,(0, 3, root),(3, 1, advmod),(3, 2, nmod))","(0-0-0,S-D-S,(0, 3, root),(3, 1, advmod),(3, 2, nmod))",D-S-S,"(1, 3, 2, 0)","((0, 3, root),(3, 1, advmod),(3, 2, nmod))","(S-D-S,(0, 3, root),(3, 1, advmod),(3, 2, nmod))","(0-0-0,S-D-S,(0, 3, root),(3, 1, advmod),(3, 2, nmod))"
44993,Text(text='Laulupeo'),1,2024-03-10T01:35,4265502,224097,"(1430, 1538)",aja_EPL,nmod_phrase,"(0, 8)",0,...,H,"(1,

In [175]:
df_shuffled = sklearn.utils.shuffle(df_10000).reset_index(drop=True)

In [170]:
test_500 = []

for idx, row in df_shuffled.iterrows():
    if row['phrase_length'] > 1:
        test_500.append(row['phrase'])
    if len(test_500) == 500:
        break

In [171]:
start = time.time()

for text in test_500:
    pattern_tagger.tag(text)
    
print(f"500 fraasi märgendamiseks kulus PhrasePatternTaggeril {time.time()-start} sekundit")

500 fraasi märgendamiseks kulus PhrasePatternTaggeril 8.387112617492676 sekundit


In [176]:
test_600 = []

for idx, row in df_shuffled.iterrows():
    if row['phrase_length'] > 1:
        test_600.append(row['phrase'])
    if len(test_600) == 600:
        break

In [177]:
start = time.time()

for text in test_600:
    pattern_tagger.tag(text)
    
print(f"600 fraasi märgendamiseks kulus PhrasePatternTaggeril {time.time()-start} sekundit")

600 fraasi märgendamiseks kulus PhrasePatternTaggeril 4.846220254898071 sekundit
